In [1]:
import torch

from pulser_diff.backend import TorchEmulator
from pulser_diff.pulser import Register, Sequence, Pulse
from pulser_diff.pulser.waveforms import ConstantWaveform
from pulser_diff.pulser.devices import MockDevice
from pulser_diff.pulser_simulation import SimConfig


In [4]:
N = 2

reg = Register.square(N, torch.tensor([8.0]), prefix="q")
seq = Sequence(reg, MockDevice)
seq.declare_channel("rydberg_global", "rydberg_global")
seq.add(Pulse(
            amplitude=ConstantWaveform(500, torch.tensor([5.0])),
            detuning=ConstantWaveform(500, torch.tensor([1.0], requires_grad=True)),
            phase=0.0
            ),
        "rydberg_global")

config = SimConfig(noise="doppler", temperature=torch.tensor([1000]))

for k in range(5):
    sim = TorchEmulator.from_sequence(seq)
    sim.set_config(config)
    print(sim._hamiltonian._doppler_detune)
    sim_results = sim.run(time_grad=True, solver="dq_me")
    print(sim_results.states[-1][0,0])

{'q0': tensor(0.7898), 'q1': tensor(2.2329), 'q2': tensor(-2.5670), 'q3': tensor(3.7050), 'q4': tensor(-1.6396), 'q5': tensor(0.4744), 'q6': tensor(-3.0174), 'q7': tensor(-2.2278), 'q8': tensor(-4.3640)}
tensor(-9.1854e-08+7.5387e-07j, grad_fn=<SelectBackward0>)
{'q0': tensor(2.2256), 'q1': tensor(-1.6533), 'q2': tensor(-2.1225), 'q3': tensor(-1.2937), 'q4': tensor(3.5540), 'q5': tensor(-0.1639), 'q6': tensor(-0.4443), 'q7': tensor(0.1061), 'q8': tensor(-2.1424)}
tensor(-4.5518e-07-3.8570e-07j, grad_fn=<SelectBackward0>)
{'q0': tensor(0.0635), 'q1': tensor(4.5877), 'q2': tensor(-0.8059), 'q3': tensor(-1.0875), 'q4': tensor(1.7961), 'q5': tensor(0.7814), 'q6': tensor(-2.0382), 'q7': tensor(0.9599), 'q8': tensor(0.8329)}
tensor(-1.1910e-06-9.6350e-07j, grad_fn=<SelectBackward0>)
{'q0': tensor(-0.0547), 'q1': tensor(-0.5456), 'q2': tensor(0.1845), 'q3': tensor(-1.0239), 'q4': tensor(-2.4785), 'q5': tensor(-0.8654), 'q6': tensor(2.0183), 'q7': tensor(-2.1343), 'q8': tensor(-0.7415)}
tensor